<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week09/FastAPI_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install fastapi uvicorn
! pip install pyngrok -q

### **[ngrok API Key 발급받기](https://ngrok.com/)**
<img src='https://velog.velcdn.com/images/kya754/post/6890ccfe-7a49-40cb-bf99-d51807f8f450/68747470733a2f2f6e67726f6b2e636f6d2f7374617469632f696d672f6f766572766965772e706e67.png'>

In [ ]:
from fastapi import FastAPI
from pyngrok import ngrok
import uvicorn
import os
import threading

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}


################### ngrok #########################
from google.colab import userdata
NGROK_API_KEY = userdata.get('ngrok')
print(NGROK_API_KEY[::2])
ngrok.set_auth_token(NGROK_API_KEY)

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    # Open a HTTP tunnel on the default port 8000
    http_tunnel = ngrok.connect(8000)
    print(f"Public URL: {http_tunnel.public_url}")

    # Run uvicorn in a separate thread
    uvicorn_thread = threading.Thread(target=run_uvicorn)
    uvicorn_thread.start()

2c4ebpOXVlOcPt58UJ9CoevJ5
Public URL: https://05e529c68f81.ngrok-free.app


In [9]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI
from pyngrok import ngrok
import threading
import time

from google.colab import userdata
your_auth_token_here = userdata.get('ngrok')

# FastAPI 애플리케이션 인스턴스 생성
app = FastAPI(title="FastAPI with ngrok", description="Lightning AI에서 실행되는 FastAPI 서버")

ngrok.set_auth_token(your_auth_token_here)

# 루트 경로('/')에 대한 GET 요청 핸들러
@app.get("/")
def home():
    return {
        "message": "안녕하세요! FastAPI가 성공적으로 실행되고 있습니다!",
        "status": "running",
        "platform": "Lightning AI"
    }

@app.get("/health")
def health_check():
    return {"status": "healthy", "service": "FastAPI"}

# Uvicorn 서버를 비동기적으로 실행하기 위한 설정
nest_asyncio.apply()

def run_server():
    """별도 스레드에서 FastAPI 서버 실행"""
    print("FastAPI 서버를 시작합니다...")
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info", reload=False)

def setup_ngrok():
    """ngrok 터널 설정"""
    try:
        # ngrok authtoken 설정 (필요한 경우)
        # ngrok.set_auth_token("your_auth_token_here")  # 실제 토큰으로 교체하세요

        print("ngrok 터널을 생성합니다...")

        # ngrok 터널 생성
        public_url = ngrok.connect(8000, "http")

        print("=" * 50)
        print(f"🎉 성공! 외부에서 접근 가능한 URL: {public_url}")
        print("=" * 50)
        print("사용 가능한 엔드포인트:")
        print(f"  • 홈페이지: {public_url}/")
        print(f"  • 헬스체크: {public_url}/health")
        print(f"  • API 문서: {public_url}/docs")
        print(f"  • ReDoc: {public_url}/redoc")
        print("=" * 50)

        return public_url

    except Exception as e:
        print(f"ngrok 설정 중 오류 발생: {e}")
        print("\n해결 방법:")
        print("1. ngrok 계정이 있는지 확인하세요 (https://ngrok.com/)")
        print("2. authtoken을 설정하세요:")
        print("   ngrok.set_auth_token('your_token_here')")
        print("3. 또는 Lightning AI의 포트 포워딩을 사용하세요")
        return None

if __name__ == "__main__":
    # 서버를 별도 스레드에서 시작
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # 서버가 시작될 때까지 잠시 대기
    print("서버 시작을 기다리는 중...")
    time.sleep(3)

    # ngrok 터널 설정
    public_url = setup_ngrok()

    if public_url:
        print(f"\n서버가 성공적으로 실행되었습니다!")
        print("서버를 중지하려면 Ctrl+C를 누르세요...")

        try:
            # 서버가 계속 실행되도록 유지
            server_thread.join()
        except KeyboardInterrupt:
            print("\n서버를 종료합니다...")
            ngrok.disconnect(public_url)
            ngrok.kill()
    else:
        print("\nngrok 설정에 실패했습니다. Lightning AI 포트 포워딩을 사용하세요.")
        print("포트 8000이 포워딩되어 있는지 확인하세요.")

FastAPI 서버를 시작합니다...
서버 시작을 기다리는 중...


INFO:     Started server process [377]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


ngrok 터널을 생성합니다...
🎉 성공! 외부에서 접근 가능한 URL: NgrokTunnel: "https://a7524d89322d.ngrok-free.app" -> "http://localhost:8000"
사용 가능한 엔드포인트:
  • 홈페이지: NgrokTunnel: "https://a7524d89322d.ngrok-free.app" -> "http://localhost:8000"/
  • 헬스체크: NgrokTunnel: "https://a7524d89322d.ngrok-free.app" -> "http://localhost:8000"/health
  • API 문서: NgrokTunnel: "https://a7524d89322d.ngrok-free.app" -> "http://localhost:8000"/docs
  • ReDoc: NgrokTunnel: "https://a7524d89322d.ngrok-free.app" -> "http://localhost:8000"/redoc

서버가 성공적으로 실행되었습니다!
서버를 중지하려면 Ctrl+C를 누르세요...


In [ ]:
!kill $(lsof -t -i:8000)

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
